In [2]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np

db_username = 'root'
db_password = 'root'
db_host = 'localhost' 
db_name = 'steam_data'

engine = create_engine(f'mysql+mysqlconnector://{db_username}:{db_password}@{db_host}/{db_name}')


In [3]:
query = 'select * from steamgen'
df = pd.read_sql(query, engine)

In [4]:
df['Tag1']=df['Tag1'].str.strip()
df['Tag2']=df['Tag2'].str.strip()
df['Price']=df['Price'].str.strip()

In [5]:
df.isnull().sum()

Title          744
RAM           3329
Size          4449
Discount     34370
Price         1928
DLC              0
Tag1           744
Tag2           744
Year           759
ReviewNum     3052
Image          744
dtype: int64

In [6]:
df['Price'] = df['Price'].str.replace('Free To Play','0')
df['Price'] = df['Price'].str.replace('Free','0')
df['Price'] = df['Price'].str.replace(r'[₹,]','',regex=True).str.strip()


In [7]:
df['RAM'] = df['RAM'].str.extract(r'(\d+)')

In [8]:
df['Discount'] = df['Discount'].str.extract(r'(\d+)')
df['Discount'].fillna(0, inplace=True)

In [9]:
df = df[df['RAM'].isin(['1','2','3','4','8','6','12','16','1024','512'])]

In [10]:
df['Size'] = df['Size'].str.extract(r'(\d+\s*(?:GB|MB))')

In [11]:
df['Year'] = df['Year'].str.replace('Coming soon','2025')
df['Year'] = df['Year'].str.replace('Q4 2024','2024')

In [12]:
df = df[df['Year']!='To be announced']

In [13]:
df['Year'] = df['Year'].str.extract(r'(\d{4})')

In [14]:
df['Year'] = df['Year'].fillna('2019').astype('int')

In [15]:
df['Positivity_Percentage'] = df['ReviewNum'].str.extract(r'(\d+)%')
df['ReviewNum'] = df['ReviewNum'].str.extract(r'the\s([\d,]+)\suser')
df['ReviewNum'] = df['ReviewNum'].str.replace(',','')

In [16]:
df['Positivity_Percentage'] = pd.to_numeric(df['Positivity_Percentage'],errors='coerce')
df['ReviewNum'] = pd.to_numeric(df['ReviewNum'],errors='coerce')

In [17]:
df['Price'] = pd.to_numeric(df['Price'],errors='coerce')
df['Price'] = df['Price'].fillna(df['Price'].median())

In [18]:
df = df.drop_duplicates(keep='first').reset_index().drop(columns='index')

In [19]:
df['Size'] = df['Size'].str.replace(' GB','000')
df['Size'] = df['Size'].str.replace(' MB','')
df['Size'] = pd.to_numeric(df['Size'],errors='coerce')
df['Size'].fillna(df['Size'].mean(), inplace=True)

In [20]:
df['Discount'] = df['Discount'].astype('int')

In [21]:
df['RAM'] = df['RAM'].astype('float')
df['RAM'].apply(lambda x : x/1024 if x in [1024,512] else x).value_counts()

RAM
4.0     8843
2.0     6935
8.0     5374
1.0     4477
0.5     1983
6.0      762
16.0     562
3.0      282
12.0     145
Name: count, dtype: int64

In [22]:
df[['Discount','Price']] = pd.DataFrame(df[['Discount','Price']].apply(lambda row: (100, row['Price']) if (row['Discount'] == 0 and row['Price'] == 0) else (row['Discount'], row['Price']), axis=1).tolist(),index=df.index)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29363 entries, 0 to 29362
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Title                  29363 non-null  object 
 1   RAM                    29363 non-null  float64
 2   Size                   29363 non-null  float64
 3   Discount               29363 non-null  float64
 4   Price                  29363 non-null  float64
 5   DLC                    29363 non-null  object 
 6   Tag1                   29363 non-null  object 
 7   Tag2                   29363 non-null  object 
 8   Year                   29363 non-null  int32  
 9   ReviewNum              21749 non-null  float64
 10  Image                  29363 non-null  object 
 11  Positivity_Percentage  21749 non-null  float64
dtypes: float64(6), int32(1), object(5)
memory usage: 2.6+ MB


In [25]:
df.dropna(how='any',inplace=True)
df = df.loc[df['Price']<5000]

In [26]:
# Normalization function
def normalize_column(column, new_min, new_max):
    col_min = column.min()
    col_max = column.max()
    return new_min + (column - col_min) * (new_max - new_min) / (col_max - col_min)


df['Normalized'] = normalize_column(df['ReviewNum'],0,1)

In [27]:
def rev(col):
    if col>0.00005:
        return "Positive"
    elif col<0.000015:
        return "Negative"
    else:
        return "Mixed"
    
df['Reviews'] = df['Normalized'].apply(rev)

In [28]:
df['Total_revenue'] = df['Price'] * df['ReviewNum']
df.drop_duplicates(subset='Title',keep='first',inplace=True)

In [29]:
df.sample(20)

,Title,RAM,Size,Discount,Price,DLC,Tag1,Tag2,Year,ReviewNum,Image,Positivity_Percentage,Normalized,Reviews,Total_revenue
2073,孤独之旅 Lonely journey,4.0,1000.000000,100.0,0.0,Yes,Adventure,Platformer,2024,17.0,https://shared.akamai.steamstatic.com/store_it...,35.0,0.000007,Negative,0.0
23431,Roskis: A Boozy Beginning,2.0,200.000000,100.0,0.0,Yes,Adventure,RPG,2021,21.0,https://shared.akamai.steamstatic.com/store_it...,95.0,0.000012,Negative,0.0
4487,Rose and Cross,8.0,3000.000000,0.0,200.0,No,Adventure,First-Person,2024,28.0,https://shared.akamai.steamstatic.com/store_it...,82.0,0.000019,Mixed,5600.0
16635,Starship Inspector,1.0,250.000000,0.0,165.0,Yes,Strategy,Adventure,2021,44.0,https://shared.akamai.steamstatic.com/store_it...,56.0,0.000036,Mixed,7260.0
10728,DRAMAtical Murder,4.0,6000.000000,0.0,880.0,No,LGBTQ+,Sexual Content,2021,2721.0,https://shared.akamai.steamstatic.com/store_it...,96.0,0.002870,Positive,2394480.0
18669,Blue Rider,2.0,650.000000,0.0,250.0,Yes,Action,Indie,2016,46.0,https://shared.akamai.steamstatic.com/store_it...,86.0,0.000038,Mixed,11500.0
27218,Chase Survivors,4.0,320.000000,80.0,31.0,Yes,Post-apocalyptic,Action Roguelike,2023,26.0,https://shared.akamai.steamstatic.com/store_it...,80.0,0.000017,Mixed,806.0
18398,European Ship Simulator,4.0,2700.000000,0.0,565.0,Yes,Simulation,Open World,2016,572.0,https://shared.akamai.steamstatic.com/store_it...,41.0,0.000595,Positive,323180.0
1404,Monster Jam Steel Titans 2,4.0,20000.000000,0.0,909.0,Yes,Racing,Sports,2021,261.0,https://shared.akamai.steamstatic.com/store_it...,78.0,0.000266,Positive,237249.0
10169,Champions Online,2.0,4037.377791,0.0,250.0,No,Free to Play,Superhero,2009,4093.0,https://shared.akamai.steamstatic.com/store_it...,71.0,0.004322,Positive,1023250.0


In [31]:
df.to_sql('steamout', con=engine, index=False, if_exists='replace')

-1